## Importing the libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime

In [ ]:
stream= open("spotify/spotify4.yaml")
spotify_details = yaml.safe_load(stream)
auth_manager = SpotifyClientCredentials(client_id=spotify_details['Client_id'],
                                        client_secret=spotify_details['client_secret'])
sp = spotipy.client.Spotify(auth_manager=auth_manager)

In [ ]:
def loop_slices(path, num_slices=20):
  cnt = 0
  cnt1 = 0
  mpd_playlists = []
  unique_tracks= pd.DataFrame()
  filenames = os.listdir(path)
  for fname in tqdm(sorted(filenames, key=len)):
    if fname.startswith("mpd.slice.") and fname.endswith(".json"):
      cnt += 1
      fullpath = os.sep.join((path, fname))
      f = open(fullpath)
      js = f.read()
      f.close()
      current_slice = json.loads(js)
      # Create a list of all playlists
      for playlist in current_slice['playlists']:
        cnt1 +=1
        mpd_playlists.append(playlist)
        if cnt1 == 1000:
          cnt1=0
          temp=pd.DataFrame(mpd_playlists)
          temp=temp.explode('tracks')
          temp=pd.DataFrame(temp['tracks'].apply(pd.Series))
          unique_tracks=pd.concat([unique_tracks,temp],axis=0)
          unique_tracks.drop_duplicates(subset=['track_uri'],inplace=True)
          mpd_playlists = []
      if cnt == num_slices:
        break
  return unique_tracks
# Path where the json files are extracted
path = './data/'

In [ ]:
import os
import json
from tqdm import tqdm

def gen_playlists(path, out):
    cnt = 0
    filenames = os.listdir(path)
    
    # Ensure the output directory exists
    output_dir = os.path.join(out)
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, "playlist.csv")
    
    with open(output_file, 'w') as f:
        f.write("playlist_id\ttrack_uris\n")
    
    for fname in tqdm(sorted(filenames, key=len)):
        if fname.startswith("mpd.slice.") and fname.endswith(".json"):
            cnt += 1
            fullpath = os.path.join(path, fname)
            
            try:
                with open(fullpath, 'r') as f:
                    current_slice = json.load(f)
                
                # Process each playlist
                for playlist in current_slice['playlists']:
                    track_uris = [track['track_uri'].replace("spotify:track:", '') for track in playlist['tracks']]
                    playlist_id = playlist['pid']
                    
                    # Write to file
                    with open(output_file, 'a') as f:
                        f.write(f"{playlist_id}\t{track_uris}\n")
            
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from file {fname}: {e}")
                continue
    
    print(f"Processed {cnt} files.")

# Path where the json files are extracted
path = 'data/spotify_million_playlist_dataset/data'

# Call the function
playlists = gen_playlists(path, "data")


In [ ]:
playlist = pd.read_csv('data/playlist.csv', sep='\t')

In [ ]:
tracks = set()
for track_uris in playlist['track_uris'].apply(eval):
    tracks.update(track_uris)

In [ ]:
df = pd.read_csv('data/1m.csv')
df_list = np.array_split(df, 100)

In [ ]:
for dataframe in df_list:
  dataframe["track_uri"] = dataframe["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
  dataframe["artist_uri"] = dataframe["artist_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
  dataframe["album_uri"] = dataframe["album_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])

In [ ]:
cols = ['track_uri', 'artist_uri', 'album_uri']
new_df_list = []
for dataframe in df_list:
  new_dataframe = dataframe[cols]
  new_df_list.append(new_dataframe)

# Feature extraction

Using the Spotify API for Feature Extraction and Saving Results to a CSV File and Errors to a Log File

I was using SP.track first, but I realised that it would take a lot of time and I would have to counter a lot of Api rate limits, so I used SP.tracks and SP.artists instead. They accept lists with a 50-URI maximum and handle them in a single request, so it took a lot less time.

In [ ]:
count = 0
for dataframe in tqdm(new_df_list[0:], total=len(new_df_list[0:])):
  print(count)
  t_uri=dataframe["track_uri"].unique()
  a_uri=dataframe["artist_uri"].unique()
  f = open('final_data/audio_features_sample_'+str(count)+'.csv','a')
  e=0
  for i in range(0,len(t_uri),100):
      try:
       track_feature = sp.audio_features(t_uri[i:i+100])
       track_df = pd.DataFrame(track_feature)
       csv_data = track_df.to_csv(header=True,index=False)
       f.write(csv_data)
      except Exception as error:
          e+=1
          r = open("audio_features_log_"+str(count)+".txt", "a")
          r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+": "+str(error)+'\n')
          r.close()
          time.sleep(1)
          continue
  count+=1
  r = open("audio_features_log_"+str(count)+".txt", "a")
  r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+" _________________________ "+"Total Number Of Errors : "+str(e)+" _________________________ "+'\n')
  r.close()
  f.close()